In [ ]:
from mediapipe.python.solutions.drawing_utils import _normalized_to_pixel_coordinates
from sklearn.model_selection import train_test_split
from imutils import paths

import cv2
import mediapipe as mp
import numpy as np
import matplotlib.pylab as plt
import os

import tensorflow as tf
from tensorflow import keras

In [ ]:
print("[INFO] loading images...")
imagePaths =list(paths.list_images("imagens/")) # give the dataset path
data = []
labels = []

for imagePath in imagePaths:
  label = imagePath.split('/')[-2]
  image = cv2.imread(imagePath)
  image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
  image = cv2.resize(image, (128, 128))
  label = int(label)
  data.append(image)
  labels.append(label)
data = np.array(data) / 255.0
labels = np.array(labels)
n_labels = len(labels)

(train_images, test_images, train_labels, test_labels) = train_test_split(data, labels, 
                                                                          test_size=0.20, random_state=42,
																																					stratify=labels)
class_names = ['Saudável', 'Varíola do Macaco']

model = keras.Sequential([
    keras.layers.Flatten(input_shape=(128, 128)),
    keras.layers.Dense(2048, activation='relu'),
    keras.layers.Dense(2, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(train_images, train_labels, epochs=25)

test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)

print("\nTest accuracy: {:5.2f}%".format(test_acc * 100))

In [6]:
webcam = cv2.VideoCapture(1)

reconhecimento_rosto = mp.solutions.face_detection
#desenho = mp.solutions.drawing_utils
reconhecedor_rosto = reconhecimento_rosto.FaceDetection()

while webcam.isOpened():
    validacao, frame = webcam.read()
    if not validacao:
        break
    imagem = frame
    lista_rostos = reconhecedor_rosto.process(imagem)
    
    if lista_rostos.detections:
        for rosto in lista_rostos.detections:
            #desenho.draw_detection(imagem, rosto)
            #image = imagem
            #output = rosto.copy()
            #circles = cv2.HoughCircles(rosto, cv2.HOUGH_GRADIENT, 1.2, 100)

            if rosto is not None:
                image_rows, image_cols, _ = imagem.shape
                location = rosto.location_data
                bounding_box = location.relative_bounding_box

                rect_start_point = _normalized_to_pixel_coordinates(
                    bounding_box.xmin, bounding_box.ymin, image_cols, image_rows
                )
                rect_end_point = _normalized_to_pixel_coordinates(
                    bounding_box.xmin + bounding_box.width,
                    bounding_box.ymin + bounding_box.height,
                    image_cols, image_rows
                )

                if rect_start_point is not None and rect_end_point is not None:
                    xleft, ytop = rect_start_point
                    xright, ybottom = rect_end_point
                
                img = imagem[ytop: ybottom, xleft: xright]
                cv2.imshow("Rosto Encontrado", img)

                if img is not None:
                    aux = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
                    aux = cv2.resize(aux, (128, 128))
                    aux = (np.expand_dims(aux, 0))
                    previsao = model.predict(aux)
                    r = np.argmax(previsao[0])
                    resultado = class_names[r]
                    #certeza = predictions[0][r] * 100
                    #resultado = resultado + " ({:0.2f}%)".format(certeza)

                    cv2.imshow(resultado, cv2.resize(aux, (512, 512)))
                    
                #if img is not None:
                    #if cv2.waitKey(5) == 27: #ESC
                        #fotoRosto = img
                        #webcam.release()
                        #break
    
    #cv2.imshow("Rostos na sua webcam", imagem)
    #if cv2.waitKey(5) == 27:
        #break

webcam.release() 
cv2.destroyAllWindows() 

In [7]:
while True:
    fotoRosto = cv2.resize(fotoRosto, (128, 128))
    #cv2.imshow("Foto", fotoRosto)
    cv2.imshow("Foto", cv2.cvtColor(fotoRosto, cv2.COLOR_BGR2GRAY))
    cv2.waitKey(0)
    break
cv2.destroyAllWindows()

Carregando Nomes

In [ ]:
#class_names = ["Saudavel", "Variola do Macaco"]

#model = tf.keras.models.load_model("modelo/")
#model.load_weights("checkpoint/")

#model.summary()

Salvando Rosto

In [ ]:
fotoRosto = cv2.cvtColor(fotoRosto, cv2.COLOR_BGR2GRAY)
fotoRosto = cv2.resize(fotoRosto, (128, 128))

Prever Resultado

In [12]:
cv2.imwrite("foto.png", fotoRosto)
aux = cv2.imread("foto.png")
aux = (np.expand_dims(aux, 0))
os.remove("foto.png")

previsao = model.predict(aux)

r = np.argmax(previsao[0])
resultado = class_names[r]
certeza = predictions[0][r] * 100
resultado = resultado + " ({:0.2f}%)".format(certeza)

1/1 [==============================] - 0s 416ms/step


In [17]:
fotoRosto = cv2.resize(fotoRosto, (256,256))
cv2.imshow(resultado, fotoRosto)
cv2.waitKey(0)
cv2.destroyAllWindows()

Exibir Resultado na Saída

In [ ]:
plt.title(resultado)
plt.axis("off")
plt.imshow(fotoRosto, cmap="gray")
plt.show()